# Here I follows the demo from PennyLane for building the H3+ molecule using VQE algorithm

In [7]:
#!pip install jax
#!pip install aiohttp fsspec h5py
#!pip install pennylane-catalyst

ERROR: Could not find a version that satisfies the requirement pennylane-catalyst (from versions: none)
ERROR: No matching distribution found for pennylane-catalyst


In [8]:
import json
import pennylane as qml
import pennylane.numpy as np
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update('jax_enable_x64', True)

In [9]:
dataset = qml.data.load('qchem', molname="H3+")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)

print(f"qubits: {qubits}")

qubits: 6


In [10]:
from jax import numpy as jnp

# Define the device, using lightning.qubit device
dev = qml.device('default.qubit', wires=qubits)


hf = np.array(dataset.hf_state)

@qml.qnode(dev, diff_method="adjoint")
def cost(params):
    qml.BasisState(hf, wires=range(qubits))
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])
    return qml.expval(H)

In [11]:
print(hf)

[1 1 0 0 0 0]


In [12]:
init_params = np.array([0.0, 0.0])
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 10

params = init_params

for n in range(10):
    params, prev_energy = opt.step_and_cost(cost, params)
    print(f"--- Step: {n}, Energy: {cost(params):.8f}")

print(f"Final angle parameters: {params}")

--- Step: 0, Energy: -1.25144067
--- Step: 1, Energy: -1.25750443
--- Step: 2, Energy: -1.26014791
--- Step: 3, Energy: -1.26129611
--- Step: 4, Energy: -1.26179425
--- Step: 5, Energy: -1.26201033
--- Step: 6, Energy: -1.26210406
--- Step: 7, Energy: -1.26214474
--- Step: 8, Energy: -1.26216238
--- Step: 9, Energy: -1.26217005
Final angle parameters: [0.16684898 0.16735496]
